## Import Libraries

In [1]:
import pandas as pd
import tensorflow as tf
from keras import Sequential, Model
from keras.layers import Dense, Conv2D, Reshape, UpSampling2D, \
    BatchNormalization, Activation, Input, LeakyReLU, Dropout, Flatten, Conv2DTranspose, concatenate
from tensorflow.keras.optimizers import Adam
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from scipy.linalg import sqrtm
from skimage.transform import resize
import os

## Model Architecture

In [2]:
class CGAN:
    def __init__(self, img_rows, img_cols, channels, num_classes, z, discriminator_steps=2, generator_steps=3):
        self.img_rows = img_rows
        self.img_cols = img_cols
        self.channels = channels
        self.num_classes = num_classes
        self.latent_dim = z
        self.discriminator_steps = discriminator_steps
        self.generator_steps = generator_steps
        self.img_shape = (img_rows, img_cols, channels)
        self.build_and_compile_models()
    
    def build_generator(self):
        """Build a Generator Model"""
        inputs = Input(shape=(self.latent_dim,))
        labels = Input(shape=(self.num_classes,))
        image_size = self.img_rows
        
        # Concatenate noise and label
        x = concatenate([inputs, labels], axis=1)
        x = Dense(image_size // 4 * image_size // 4 * 128)(x)
        x = Reshape((image_size // 4, image_size // 4, 128))(x)

        for filters in [128, 64, 32, self.channels]:
            strides = 2 if filters > 32 else 1
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            x = Conv2DTranspose(filters=filters, kernel_size=5, strides=strides, padding='same')(x)

        x = Activation('tanh')(x)
        generator = Model([inputs, labels], x, name='generator')
        generator.summary()
        return generator
    
    def build_discriminator(self):
        """Build a Discriminator Model"""
        inputs = Input(shape=self.img_shape)
        labels = Input(shape=(self.num_classes,))
        image_size = self.img_rows
        
        # Embed the labels
        y = Dense(image_size * image_size)(labels)  # Ensure this matches the flattened size
        y = Reshape((image_size, image_size, 1))(y)  # Correct shape
        
        x = concatenate([inputs, y])

        for filters in [32, 64, 128, 256]:
            strides = 2 if filters != 256 else 1
            x = Conv2D(filters=filters, kernel_size=5, strides=strides, padding='same')(x)
            x = LeakyReLU(alpha=0.2)(x)
            x = Dropout(0.4)(x)

        x = Flatten()(x)
        x = Dense(1)(x)
        x = Activation('sigmoid')(x)
        discriminator = Model([inputs, labels], x, name='discriminator')
        discriminator.summary()
        return discriminator


    def build_and_compile_models(self):
        """Build and compile Generator and Discriminator models"""
        # Build discriminator model
        self.discriminator = self.build_discriminator()
        optimizer = Adam(learning_rate=0.0002, beta_1=0.5)
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        
        # Build generator model
        self.generator = self.build_generator()

        # Build adversarial model
        self.discriminator.trainable = False
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(self.num_classes,))
        img = self.generator([noise, label])
        validity = self.discriminator([img, label])
        self.combined = Model([noise, label], validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

### Loading weights

In [3]:
# Initialize the CGAN
cgan = CGAN(img_rows=28, img_cols=28, channels=1, num_classes=26, z=100)

# Load the weights into the generator, discriminator, or combined model
cgan.combined.load_weights('cgan_train_weights/cgan_Epoch_4900_combined.h5')

Model: "discriminator"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 26)]         0           []                               
                                                                                                  
 dense (Dense)                  (None, 784)          21168       ['input_2[0][0]']                
                                                                                                  
 input_1 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 reshape (Reshape)              (None, 28, 28, 1)    0           ['dense[0][0]']                  
                                                                                      

### Generate Images

## Deployment to render process (show render link)